# Workshop #5


Welcome to workshop #5! In this workshop, we are going to talk about a classic classification model: K-Nearest Neighbors. We're also going to talk about different ways to evaluate an ML model's performance. 

Lets jump in!

Start off my importing our data:

In [ ]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/uclaacmai/ML_Practical/master/cleaned_titanic_data.csv"

titanic_data = pd.read_csv(url)
titanic_data.drop(titanic_data.columns[0], inplace=True, axis = 1)
print(titanic_data)

     Survived  Pclass  Sex   Age  SibSp  Parch     Fare  port_Q  port_S
0           0       3    0  22.0      1      0   7.2500       0       1
1           1       1    1  38.0      1      0  71.2833       0       0
2           1       3    1  26.0      0      0   7.9250       0       1
3           1       1    1  35.0      1      0  53.1000       0       1
4           0       3    0  35.0      0      0   8.0500       0       1
..        ...     ...  ...   ...    ...    ...      ...     ...     ...
884         0       2    0  27.0      0      0  13.0000       0       1
885         1       1    1  19.0      0      0  30.0000       0       1
886         0       3    1  28.0      1      2  23.4500       0       1
887         1       1    0  26.0      0      0  30.0000       0       0
888         0       3    0  32.0      0      0   7.7500       1       0

[889 rows x 9 columns]


And now we split our data into train and test groups:

In [ ]:
from sklearn.model_selection import train_test_split

labels = titanic_data['Survived']
features = titanic_data.drop(columns = ['Survived'])

features.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,port_Q,port_S
0,3,0,22.0,1,0,7.2500,0,1
1,1,1,38.0,1,0,71.2833,0,0
2,3,1,26.0,0,0,7.9250,0,1
3,1,1,35.0,1,0,53.1000,0,1
4,3,0,35.0,0,0,8.0500,0,1


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=42)
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,port_Q,port_S
707,1,1,22.0,0,0,151.5500,0,1
239,3,1,28.0,1,0,14.4542,0,0
381,3,0,32.0,0,0,7.9250,0,1
791,3,1,28.0,8,2,69.5500,0,1
682,3,0,14.0,5,2,46.9000,0,1


Let's verify that the train/test split actually worked:

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print(titanic_data.shape)
print(len(x_train) + len(x_test))

(711, 8)
(178, 8)
(711,)
(178,)
(889, 9)
889


### K Nearest Neighbors

K Nearest Neighbors is another ML algorithm that is very useful for classification tasks like ours! Let's say you're about to go vote in the upcoming Democratic Primaries, but don't know too much about the candidates. You decide to ask the five people near you who they are voting for. Three of these people say they are voting for Candidate A, and two say they are voting for Candidate B. You decide that since more people decided to vote for Candidate A, you will too. 

A K Nearest Neighbors model works in a similar way. We graph datapoints based on their features, and then for each data point, assign it the value of those nearest to it. Here's a visual example:

In [ ]:
from IPython.display import Image

img_url = "https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1531424125/KNN_final1_ibdm8a.png"
Image(url=img_url)

Let's start with the assumption that our we are looking at the three nearest neighbors, so k is equal to three. In the graph on the top left, we have plotted the classified data we have, and have also placed the data point we want to classify in yellow. Next, in the top right graph, we measure the distance of our new data point to its nearest neighbors. In the bottom graph, we can see that of the three closest nearest neighbors to our new datapoint, two are Class B, and one is Class A. Since the majority of the three closest neigbors are Class B, we will label our new datapoint as Class B as well.

In [ ]:
img_url = "https://miro.medium.com/max/1300/0*Sk18h9op6uK9EpT8."
Image(url=img_url)

Questions to think about

1. What would we classify our point in green if K=1? If K=3? K=8? K=20?

2. Is it possible for us to obtain 0 training error?

3. Why would you almost always have a gap between training and testing error?

Let's implement KNN using scikit learn:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize a vanilla kNN model
knn_model = KNeighborsClassifier()
# Fit it to the training data
knn_model.fit(x_train, y_train)

print("Train accuracy: " + str(knn_model.score(x_train, y_train)))
print("Test accuracy: " + str(knn_model.score(x_test, y_test)))

Train accuracy: 0.8016877637130801
Test accuracy: 0.6853932584269663


We can change the number of neighbors we survey using sklearn's model with the n_neighbors parameter:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors = 3).fit(x_train, y_train)

print("Train accuracy: " + str(knn_model.score(x_train, y_train)))
print("Test accuracy: " + str(knn_model.score(x_test, y_test)))

Train accuracy: 0.8438818565400844
Test accuracy: 0.7247191011235955


Hold up, what happened here? Our train accuracy is high, but our test accuracy is extremely low. Once again, this is an example of overfitting. Unfortunately, if we don't have an adequate amount of data, a KNN is likely to overfit. This is called the curse of dimensionality.

In [ ]:
img_url = "https://www.visiondummy.com/wp-content/uploads/2014/04/dimensionality_vs_performance.png"
Image(url=img_url)

The curse of dimensionality essentially says that as we increase the number of features in our model, classifier performance will likely drop if we don't exponentially increase the amount of data. In our case, we've likely given the model too many features to handle.

Let's see what we can do about this. Let's reduce the number of features our model looks at and see if that helps.

In [ ]:
new_data = titanic_data.drop(columns = ["SibSp", "Fare", "port_Q", "port_S"])
new_data.head()


,Survived,Pclass,Sex,Age,Parch
0,0,3,0,22.0,0
1,1,1,1,38.0,0
2,1,3,1,26.0,0
3,1,1,1,35.0,0
4,0,3,0,35.0,0


Now for our train test split, as per standard procedure:

In [ ]:
features = new_data.drop(columns = ["Survived"])
labels = new_data["Survived"]

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=42)

x_train.head()

,Pclass,Sex,Age,Parch
707,1,1,22.0,0
239,3,1,28.0,0
381,3,0,32.0,0
791,3,1,28.0,2
682,3,0,14.0,2


In [ ]:
knn_model = KNeighborsClassifier(n_neighbors = 3).fit(x_train, y_train)
print("Train accuracy: " + str(knn_model.score(x_train, y_train)))
print("Test accuracy: " + str(knn_model.score(x_test, y_test)))

Train accuracy: 0.8523206751054853
Test accuracy: 0.7865168539325843


okay.....well that didn't really work. Perhaps we're choosing a poor value for n_neighbors. Let's try some parameter tuning! Here, we'll loop through different values of n_neighbors and see what works the best.

In [ ]:
# n = [1, 3, 5, 11, 21, 51, 101, 601]

n = range(1, 101, 2)

for a in n:
    knn_model = KNeighborsClassifier(n_neighbors = a).fit(x_train, y_train)
    print("Number of neighbors: " + str(a))
    print("Train accuracy: " + str(knn_model.score(x_train, y_train)))
    print("Test accuracy: " + str(knn_model.score(x_test, y_test)))
    print()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
n = range(1, 101, 2)

for a in n:
    knn_model = KNeighborsClassifier(n_neighbors = a).fit(x_train, y_train)
    print("Number of neighbors: " + str(a))
    print("Train accuracy: " + str(knn_model.score(x_train, y_train)))
    print("Test accuracy: " + str(knn_model.score(x_test, y_test)))
    print()

Number of neighbors: 1
Train accuracy: 0.8804500703234881
Test accuracy: 0.8146067415730337

Number of neighbors: 3
Train accuracy: 0.8607594936708861
Test accuracy: 0.8258426966292135

Number of neighbors: 5
Train accuracy: 0.849507735583685
Test accuracy: 0.8146067415730337

Number of neighbors: 7
Train accuracy: 0.8523206751054853
Test accuracy: 0.8033707865168539

Number of neighbors: 9
Train accuracy: 0.8466947960618847
Test accuracy: 0.8033707865168539

Number of neighbors: 11
Train accuracy: 0.8382559774964838
Test accuracy: 0.8033707865168539

Number of neighbors: 13
Train accuracy: 0.8270042194092827
Test accuracy: 0.797752808988764

Number of neighbors: 15
Train accuracy: 0.8270042194092827
Test accuracy: 0.8089887640449438

Number of neighbors: 17
Train accuracy: 0.8185654008438819
Test accuracy: 0.8258426966292135

Number of neighbors: 19
Train accuracy: 0.8171589310829818
Test accuracy: 0.8314606741573034

Number of neighbors: 21
Train accuracy: 0.8115330520393812
Test acc

What's the best way to decide the optimal choice of k?
(Hint: Is a train-test split always enough?)

It seems that the best value we have for k is 13, but the performance still isn't amazing. No worries though! This just means that the KNN algorithm is not a good choice for our titanic dataset. This sort of trial and error is very common when you're just starting off in data science or machine learning.

That being said, there are a couple of major advantages of kNN (that are kinda related). Anyone want to take a guess?
1. Training is essentially free. 
2. kNN an 'online' algorithm: can add more data points easily.

There's also a third pretty obvious one:
3. kNN is easily *interpretable*. 

In ML, you always have a trade-off. When do advantages 1 and 2 above lead to a disadvantage?

Prediction can be expensive

### Different Scenarios 

Most of the time, in the real world, the majority of values are measured with accuracy. You get scored on how accurate you were on your exam, and basketball free throws percentage is measured with accuracy, just to name a few. 

### Why Accuracy Isn't Always the Best Choice

Let's say we were designing and algorithm that identified whether or not objecting coming into Pauley Pavillion were potential civilian hazards (bombs, knives, etc). Well, most people don't bring these types of objects into Pauley Pavillion, because they know of the laws and the consequences of breaking those laws. This model, if trained on data that accurately respected the distribution of objects coming into Pauley would be accurate 99% of the time, as the vast majority of objects coming into Pauley aren't hazardous. Thus, measuring our algorithm with accuracy would be useless here.

Let's take another example. Let's say we designed an algorithm that could examine a lottery ticket and tell you whether or not it was going to be a winning ticket. If we simply wrote an algorithm that told you no every time, the algoirhtm with would have an accuracy of 99.999%. But, once again, this algoirhtm would be useless, and would not provide any actual value. Thus, once more, accuracy is not the best way to evaluate this algorithm.

There has to be a better way to do this!

### Confusion Matrix

Before we get into a few better stats to measure our models, lets define a few terms. 

'Result' below refers to the output of the model, expected result is the true value

__True Positive__: When the result and expected result are both positive

__False Positive__: When the result is positive but the expected result is negative

__True Negative__: When the result and expected result are both negative

__False Negative__: When the result is negative but the expected result is positive

Using these definitions, we can create a useful table known as a __Confusion Matrix__:

In [ ]:
from IPython.display import Image
img_url = "https://glassboxmedicine.files.wordpress.com/2019/02/confusion-matrix.png?w=816"
Image(url=img_url)

A confusion matrix essentially shows a distribution of true positives, false positives, true negatives and false negatives. They are really useful for easily looking at types of mistakes your model is making.

Try making a confusion matrix of your own of the following data:

In [ ]:
import pandas as pd

practice_data_url = "https://raw.githubusercontent.com/uclaacmai/ML_Practical/master/practiceData.csv"
data = pd.read_csv(practice_data_url)

print(data.head())

   predicted  expected
0          1         1
1          1         0
2          1         1
3          0         1
4          1         1


In [ ]:
from sklearn.metrics import confusion_matrix
arr = confusion_matrix(data['expected'], data['predicted'])
print(arr)

[[11 10]
 [13 15]]


### Precision and Recall

Using the four definitions we just learned, we can define two better measures of accuracy: __Precision__ and __Recall__:

Run the following cell to see their definitions:

In [ ]:
from IPython.display import Image
img_url = "https://miro.medium.com/max/2708/1*6NkN_LINs2erxgVJ9rkpUA.png"
Image(url=img_url)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

print(precision_score(data['expected'], data['predicted']))
print(recall_score(data['expected'], data['predicted']))

0.6
0.5357142857142857


Exercise: calculate the Precision and Recall of the practice data from above! (Hint: See if you can find a Scikit-learn function that will do the hard work for you)

### Recall Scenarios

Recall is a good choice to use when it is important to minimimze true negatives. Recall would be a good choice to use in a situation like the security one we outlined earlier. It is obviously much better to check a few extra items than to accidentally let an item that is hazardous.

### Precision Scenarios

Precision is a good choice when we want to maximize true positives. What this means is that when we say that we have a positive, we are very sure that it is actually a positive. 

An example where we want high precision is the criminal justice system. When convicting someone, we want to be 100% sure that they are guilty. It is better to let a few guilty people be free than to imprison innocent people.

### F1 Score

In most situations, however, we'd want a balance of both Precision and Recall. For example, we could easily get 100% recall by simply marking every item going into Pauley Pavillion as a hazard. In reality of course, this would be extremely unfeasable, as it waste a lot of time for security personnel at Pauley. It would be best if we could find some ideal where we maximize the number of people we catch while minimizing the amount of time wasted. 

In this sort of situation, we should use an F1 Score. Run the following cell to see the definition for an F1 score:

In [ ]:
from IPython.display import Image
img_url = "https://miro.medium.com/max/1124/1*V27Dd47fxtCB-u561I8nYQ.png"
Image(url=img_url)

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(data['expected'], data['predicted'])) 

0.5660377358490566


Clearly, an F1 score is simply a harmonic mean of Precision and Recall. Use the next cell to find the F1 Score of our practice data!

And with that, you should have enough tools to start to evaluate models better! If you're interested in more ways to evaluate your model, do some research on AUC and ROC curves. Please fill out our feedback form here: https://forms.gle/SaZXtekbbQecvnvo9

See you next week!